# Training a neural network

In questa parte vengono introdotti i modelli statistici di spacy e a come personalizzarli per le nostre esigenze

## Training and updating a model

Ci focalizziamo in particolare sulla NER. Le domande a cui risponde questo paragrafo sono *"Perchè personalizzare un modello?"* e *"Perchè non ci possiamo accontentare di un modello pre-addestrato?"*.

I modelli pre-addestrati hanno visto esempi generalisti, con la specializzazione li possiamo rendere più performanti sul nostro dominio. Questo è fondamentale e semplice quando si tratta di NER, un pò più critico se parliamo di POS-tagging e dependency parsing.

Gli step dell'addestramento sono:

* **inizializza** il modello con pesi random
* **predici** alcuni esempi con i pesi attuali (usando `nlp.update`)
* **compara** le previsioni con le etichette reali
* **calcola** come modificare i pesi per migliorare le previsioni
* **aggiorna** i pesi
* torna alla 2 e riparti

<br>

![training](https://course.spacy.io/training.png)

<br>

### Esempio

L'entity recognition prende il testo e ne predice frasi e label nel contesto, quindi per addestrare questi modelli bisogna avere:
* il testo
* le entità
* le etichette delle entità

Ogni token può fa parte di una sola entità, non possono esserci sovrapposizioni. Per mostrare le entità in SpaCy si possono usare `Doc` e `Span`, è anche molto importante mostrare al modello i termini che non sono entità.
Il nostro obiettivo è insegnare al modello a riconoscere nuove entità in contesti simili, anche se non erano presenti nei dati di addestramento.

## Volume di dati

Se si vuole aggiornare un modello pre-addestrato possono bastare centinaia o migliaia di esempi.

Se si vuole addestrare una nuova categoria ci vorranno milioni di esempi, ad esempio la pipeline di SpaCy per l'inglese è stata addestrata su due milioni di parole con etichette per NER, POS-tagging e dependency parsing.

Le etichette sono spesso create manualmente ma SpaCy forniscge il `Matcher` che ci aiuta in una creazione semi automatica.

Ci sono tre tipi di dati:
* **training**: per addestrare il modello
* **evaluation**: mai visti prima dal modello, servono per misurarne le performance
* **test**: mai visti dal modello ci aiutano a capire come si comporterebbe nel mondo reale

In [4]:
import spacy
from spacy.tokens import DocBin, span
import random

nlp = spacy.blank("en")

# Create a Doc with entity spans
doc1 = nlp("iPhone X is coming")
doc1.ents = [span.Span(doc1, 0, 2, label="GADGET")]
# Create another doc without entity spans
doc2 = nlp("I need a new phone! Any tips?")

docs = [doc1, doc2]

random.shuffle(docs)
train_docs = docs[:len(docs) // 2]
dev_docs = docs[len(docs) // 2:]

# DocBin: container to efficiently store and save Doc objects
# Create and save a collection of training docs
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("./train.spacy")
# Create and save a collection of evaluation docs
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("./dev.spacy")

Il comando convert di spaCy converte automaticamente questi file nel formato binario di spaCy. Converte anche i file JSON nel vecchio formato utilizzato da spaCy v2.

```bash
$ python -m spacy convert ./train.gold.conll ./corpus
```

## Configuring and running training

### Config

Una volta creati i dataset si può passare alla fase di addestramento. In questa fase SpaCy usa un file di configurazione `config.cfg` dove vengono indicati la pipeline di nlp con i suoi componenti e tutti gli hyperparametri. Questa modalità è utile anche per rendere riproducibile la pipeline di addestramento. Un esempio di config file:

```
[nlp]
lang = "en"
pipeline = ["tok2vec", "ner"]
batch_size = 1000

[nlp.tokenizer]
@tokenizers = "spacy.Tokenizer.v1"

[components]

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
hidden_width = 64
# And so on...
```

Se volessimo usare qualche funzione custom nella nostra pipeline possiamo inserirla utilizzando il decorator *@*.

Se non vogliamo partire da zero a creare il config file SpaCy ha una funzione che lo crea per noi:

```bash
$ python -m spacy init config ./config.cfg --lang en --pipeline ner
```

le componenti sono:
* `init config` per far partire i comando
* `./config.cfg` il file da generare
* `--lang en` la lingua del modello
* `--pipeline ner` le componenti della pipeline


Il config file può essere creato anche tramite il [quickstart widget](https://spacy.io/usage/training#quickstart).

> per visualizzare il file appena creato possiamo usare il comando `$ cat ./config.cfg`

### Run

Avendo la pipeline possiamo addestrare il modello col seguente comando:

```bash
$ python -m spacy train ./config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy
```

* `train` il comando da eseguire
* `./config.cfg` la pipeline da utilizzare
* `--output ./output` la pipeline addestrata
* `--paths.train train.spacy`, `--paths.dev dev.spacy` i dataset da utilizzare

Di seguito un'immagine di esempio della pipeline in train:

<br>

![train](https://miro.medium.com/v2/resize:fit:695/0*GR3Ibv_u4qH-tb9t.png)

<br>

* nella prima colonna le epoche di apprendimento
* nella seconda colonna ogni quanti esempi il modello fa un check
* nell'ultima l'accuratezza del modello

> L'addestramento viene eseguito finché il modello non smette di migliorare ed esce automaticamente.

### Load

Alla fine dell'addestramento vengono salvate due pipeline:

* l'ultima epoca di apprendimento `model-last`
* la migliore epoca di apprendimento `model-best`

Queste pipeline possono essere caricate normalmente come quando usiamo `spacy.load()` per `en_core_web_sm`.

```python
import spacy

nlp = spacy.load("/path/to/output/model-best")
doc = nlp("iPhone 11 vs iPhone 8: What's the difference?")
print(doc.ents)
```

### Deploy

Per rendere più semplice portare in produzione o codividere le pipeline personalizzate SpaCy mette a dispsizione il modulo `package` che permette di creare un pacchetto `.tar.gz` installabile e utilizzabile. Per creare il pacchetto:

```bash
$ python -m spacy package /path/to/output/model-best ./packages --name my_pipeline --version 1.0.0
```

per installare il pacchetto nel nostro environment

```bash
$ cd ./packages/en_my_pipeline-1.0.0
$ pip install dist/en_my_pipeline-1.0.0.tar.gz
```

per utilizzarlo in spacy

```python
nlp = spacy.load("en_my_pipeline")
```

### Best practices

Di seguito alcuni problemi in cui si può incorrere allenando un modello pre-addestrato

#### Catastrophic forgetting problem

Se addestro un modello pre-addestrato con tanti nuovi esempi di una particolare etichetta potrebbe *dimenticare* qualcosa che aveva imparato prima. La *soluzione* è di non verticalizzare eccessivamente il dataset *specializzato* su cui si raffina il modello ma includere anche altre etichette che c'erano in quello generalista.

#### Local Context

Altro problema è quello di specializzare troppo le etichette, ad esempio `ADULT_CLOATHING` e `CHILDREN_CLOATHING` potrebbero essere delle etichette molto specializzate che il modello non riesce a distinguere, meglio un generico `CLOATHING`. Successivamente si possono specializzare mediante delle regole.